# Parallel split-apply-combine

A nice property of split-apply-combine is that it is well suited to parallel computation, in the sense that the apply part can be done independently in parallel. The disadvantage is that you cannot use the quick-and-easy functionality we used above, as you need to do each part separately.

We now proceed with exercises where you

1. Use the python multiprocessing module to do the apply part in parallel.
2. Use slurm array jobs to do the apply part in parallel.


## Split-apply-combine using multiprocessing

Here we use the Pool class in the multiprocessing module, which is an easy way to parallelize simple tasks.

In [1]:
from multiprocessing import Pool
import os
import pandas as pd
import os.path
import scipy.stats as stats

d = pd.read_csv(os.path.join(pd.__path__[0], 'tests/data/iris.csv'), dtype={'Name': 'category'})

sz = os.getenv('OMP_NUM_THREADS', 1)
sz2 = os.getenv('SLURM_CPUS_PER_TASK', 1)
sz = max(sz, sz2)
print("Using ", sz, " processes.")
def f(x):
    return x[1].describe()
pool = Pool(sz)

res = pool.map(f, d.groupby('Name'))
# Calculate e.g. t-test for difference in sepallength between setosa and versicolor
rs = res[0]['SepalLength']
rv = res[1]['SepalLength']
stats.ttest_ind_from_stats(rs['mean'], rs['std'], rs['count'], rv['mean'], rv['std'], rv['count'])

Using  1  processes.


Ttest_indResult(statistic=-10.52098626754912, pvalue=8.985235037486755e-18)

## Parallel split-apply-combine with slurm array jobs

For this we need to step outside the warm confines of the jupyter notebook and face the cold, hard reality of the shell. You will need several python scripts, and several slurm batch scripts. What you need is

- A python script to read the original dataset and split it into separate datasets, and then write each dataset into a separate file. This is the *split* phase.
- A python script to read in a dataset (a split dataset generated in the previous step), and perform the operation we're interested in, and write the part to an output file. This is the *apply* phase, and can be run in parallel, once for each dataset.
- A python script that combines the results of the apply phase and generates the final result. This is the *combine* phase.
- A slurm batch script to run the split phase, and then submits jobs for the apply and combine phases. *Hint*: For the apply phases, you should use a slurm array job.
- A slurm batch script to run the combine phase. *Hint*: This job needs to use slurm job dependencies so that it only runs after the successful completion of the apply phase jobs.

The solutions to this can be found in the repo in the directory `python/split-apply-combine-parallel`